## Ejercicio 6

Los archivos **Segment_Train.csv** y **Segment_Test.csv** contienen información referida a regiones de 3x3 pixeles pertenecientes a 7 imágenes distintas. Cada una corresponde a uno de los siguientes tipos de superficie: ladrillo, cielo, follaje, cemento, ventana, camino y pasto.  

Cada región de 3x3 ha sido caracterizada por 19 atributos numéricos:  

1. **region-centroid-col:** la columna del pixel central de la región.
2. **region-centroid-row:** la fila del pixel central de la región.
3. **region-pixel-count:** el número de pixeles de la región = 9.
4. **short-line-density-5:** el resultado de un algoritmo de extracción de líneas que cuenta la cantidad de líneas de bajo contraste que atraviesan la región.
5. **short-line-density-2:** ídem anterior para líneas de alto contraste.
6. **vedge-mean:** medida del contraste entre pixeles adyacentes. Este atributo contiene el valor promedio y el siguiente la desviación. Estas medidas sirven para detectar la presencia de un eje vertical.
7. **vegde-sd:** (ver 6)
8. **hedge-mean:** ídem 6 para eje horizontal. Contiene el valor medio y el siguiente la desviación.
9. **hedge-sd:** (ver 8).
10. **intensity-mean:** El promedio calculado sobre la región de la forma (R + G + B)/3
11. **rawred-mean:** el promedio sobre la región de los valores R.
12. **rawblue-mean:** el promedio sobre la región de los valores B.
13. **rawgreen-mean:** el promedio sobre la región de los valores G.
14. **exred-mean:** Medida de exceso de color rojo: (2R - (G + B))
15. **exblue-mean:** Medida de exceso de color azul: (2B - (G + R))
16. **exgreen-mean:** Medida de exceso de color verde: (2G - (R + B))
17. **value-mean:** Transformación no lineal 3D de RGB.
18. **saturatoin-mean:** (ver 17)
19. **hue-mean:** ver 17

El atributo 20 corresponde al número de imagen de la cual fue extraída la región de 3x3. Sus valores son: 1 (ladrillo), 2 (cemento), 3 (follaje), 4 (pasto), 5 (camino), 6 (cielo), 7 (ventana).  

Entrene una red neuronal multiperceptrón para que dada una región de 3x3, representada a través de los 19 atributos indicados anteriormente, sea capaz de identificar a cuál de las 7 imágenes corresponde.  

Utilice los ejemplos del archivo **Segment_Train.csv** para entrenar y los del archivo **Segment_Test.csv** para realizar el testeo.  

Realice al menos 10 ejecuciones independientes de la configuración seleccionada para respaldar sus afirmaciones referidas a la performance del modelo.  

Fuente: **Image Segmentation Data Set:** https://archive.ics.uci.edu/ml/datasets/Image+Segmentation  

#### Entrenamiento

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing, model_selection
from sklearn.neural_network import MLPClassifier

TEST_SIZE = 0.3

DATOS_DIR   = '../../Datos/'
train_data = pd.read_csv(DATOS_DIR + 'Segment_Train.csv', header=0, names=[
    "label",
    "REGION-CENTROID-COL", "REGION-CENTROID-ROW", "REGION-PIXEL-COUNT",
    "SHORT-LINE-DENSITY-5", "SHORT-LINE-DENSITY-2",
    "VEDGE-MEAN", "VEDGE-SD", "HEDGE-MEAN", "HEDGE-SD",
    "INTENSITY-MEAN", "RAWRED-MEAN", "RAWBLUE-MEAN", "RAWGREEN-MEAN",
    "EXRED-MEAN", "EXBLUE-MEAN", "EXGREEN-MEAN",
    "VALUE-MEAN", "SATURATION-MEAN", "HUE-MEAN"
])

binarizer = preprocessing.LabelBinarizer()

X_train = np.array(train_data.iloc[:,1:])
Y_train = np.array(train_data.iloc[:, 0])
Y_train = binarizer.fit_transform(Y_train)

scaler = preprocessing.StandardScaler()
X_train  = scaler.fit_transform(X_train)

FunH = 'tanh'   # opciones: identity logistic tanh relu
ocultas = (20, 10)

alfa = 0.01
CotaError = 0.0002
MAX_ITE = 2000


modelo = MLPClassifier(max_iter=MAX_ITE, hidden_layer_sizes=ocultas, alpha=alfa,
					solver='adam', activation=FunH, tol=CotaError,
					verbose=False).fit(X_train, Y_train)

Y_pred = modelo.predict(X_train)
score = modelo.score(X_train, Y_train)

Y_it = binarizer.inverse_transform(Y_train)
Y_pred_it = binarizer.inverse_transform(Y_pred)

print('Entrenamiento:')
print('Efectividad: %6.2f%%' % (100*(Y_pred_it == Y_it).sum()/len(Y_it)) )
print('      Score: %6.2f' % (score) )

Entrenamiento:
Efectividad:  94.76%
      Score:   0.95


#### Prueba

In [4]:
test_data = pd.read_csv(DATOS_DIR + 'Segment_Test.csv', header=0, names=[
    "label",
    "REGION-CENTROID-COL", "REGION-CENTROID-ROW", "REGION-PIXEL-COUNT",
    "SHORT-LINE-DENSITY-5", "SHORT-LINE-DENSITY-2",
    "VEDGE-MEAN", "VEDGE-SD", "HEDGE-MEAN", "HEDGE-SD",
    "INTENSITY-MEAN", "RAWRED-MEAN", "RAWBLUE-MEAN", "RAWGREEN-MEAN",
    "EXRED-MEAN", "EXBLUE-MEAN", "EXGREEN-MEAN",
    "VALUE-MEAN", "SATURATION-MEAN", "HUE-MEAN"
])

X_test = np.array(test_data.iloc[:,1:])
Y_test = np.array(test_data.iloc[:, 0])
Y_test = binarizer.fit_transform(Y_test)

scaler = preprocessing.StandardScaler()
X_test  = scaler.fit_transform(X_test)

Y_pred = modelo.predict(X_test)
score = modelo.score(X_test, Y_test)

# "invierte" la transformacion binaria para obtener los nombres de las clases
Y_it = binarizer.inverse_transform(Y_test)
Y_pred_it = binarizer.inverse_transform(Y_pred)

print('Prueba:')
print('Efectividad: %6.2f%%' % (100*(Y_pred_it == Y_it).sum()/len(Y_it)) )
print('      Score: %6.2f' % (score) )

Prueba:
Efectividad:  87.95%
      Score:   0.88
